In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('../input/fer2013/fer2013.csv')


In [ ]:
width=48
height=48
DataPoints=data["pixels"].tolist()

In [ ]:
print(len(DataPoints[2]))
print(len(DataPoints[0]))

In [ ]:
X=[]
for i in DataPoints:
    x1 = [int(x) for x in i.split(' ')]
    x1=np.asarray(x1).reshape(width,height)
    X.append(x1.astype('float32'))
X = np.asarray(X)
print(X.shape)
X = np.expand_dims(X, -1)
print(X.shape)
    

In [ ]:
y = pd.get_dummies(data['emotion']).to_numpy()

print(y.shape)

In [ ]:
np.save('fdataX', X)
np.save('flabels', y)
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

In [ ]:
print("Preprocessing Done")
print("Number of Features: "+str(len(X[0])))
print("Number of Labels: "+ str(len(y[0])))
print("Number of samples in dataset:"+str(len(X)))

In [ ]:
import sys,os
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
import matplotlib.pyplot as plt

In [ ]:
labels=7
epochs=100
batch_size=64
# standarisation of x
x=X
x=x-np.mean(x,axis=0)
x/=np.std(x,axis=0)
print(x)
# values in -1 to 1
print(x.shape)
print(X.shape)

In [ ]:
for x1 in range(2):
   plt.figure(x1)
   plt.imshow(x[x1], interpolation='none', cmap='gray')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
np.save('X_test',X_test)
np.save('y_test',y_test)

# **CNN MODEL**

conv2d since 2d kernel used in convolution

In [ ]:
model=Sequential()
n=64  
# no of neurons

model.add(Conv2D(n,kernel_size=(3,3),activation='relu',input_shape=(width, height, 1)))
model.add(Conv2D(n,kernel_size=(3,3),activation='relu',padding='same'))
# Same padding means the size of output feature-maps are the same as the input feature-maps
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*n,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*n,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*n,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*n,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*n,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*n,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(2*2*2*n, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2*2*n, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2*n, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(labels,activation='softmax'))

model.summary()

# There are mainly two types of non-trainable weights:

# The ones that you have chosen to keep constant when training. This means that keras won't update these weights during training at all.
# The ones that work like statistics in BatchNormalization layers. They're updated with mean and variance, but they're not "trained with backpropagation".



In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          shuffle=True)

**100 epochs
batch size:64**

In [ ]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")
print("Saved model to disk")

In [ ]:
y_predicted=model.predict(X_test)
print(y_predicted.shape)

In [ ]:
import joblib
yh = y_predicted.tolist()
yt = y_test.tolist()
count = 0
predy=[]
truey=[]
for i in range(len(y_test)):
    yy = max(yh[i])
    yyt = max(yt[i])
    predy.append(yh[i].index(yy))
    truey.append(yt[i].index(yyt))
    if(yh[i].index(yy)== yt[i].index(yyt)):
        count+=1

acc = (count/len(y_test))*100
print(acc)
np.save('truey', truey)
np.save('predy', predy)

# joblib.dump(model, "data_transformer.joblib")
model.save('model.h5')

> **67% accurate on test data**

In [ ]:
from PIL import Image 
import cv2
from urllib.request import urlopen
from PIL import Image




img = cv2.imread("../input/myimgs/WIN_20210327_22_38_50_Pro.jpg")

print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(img.shape)
plt.imshow(img,cmap="gray")  
plt.show() 

In [ ]:
from skimage.transform import resize
from skimage import data
image = np.array(img)
image=resize(image, (48, 48,1))
x12=image
x12=x12-np.mean(x12,axis=0)
x12/=np.std(x12,axis=0)
# print(x)
plt.imshow(x12,cmap="gray")  
plt.show() 

In [ ]:
arr=model.predict(np.array([image]))

c=0
index=0
max=0.00
for x in arr[0]:
    if x > max:
        max=x
        index=c
    c+=1
    
print(index)
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)

**saving model with weights for future usage**

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink 
FileLink(r'model.h5')